In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import re
import itertools
import pandas as pd
import numpy as np

# Begining Data Collection

In [14]:
#xpath converter
def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [12]:
browser = webdriver.Chrome()
browser.get("https://www.digitalrock.de/icc_calendar.php")
time.sleep(2)
soup = BeautifulSoup(browser.page_source, "html.parser")
link  = browser.find_element_by_name('year')
# big dataframe to hold all of the information
'ID','Rank','Last Name','First Name', 'Points'
big_df = pd.DataFrame(  {'ID':[0],
                        'Rank':[0],
                        'Last Name':[0],
                        'First Name':[0],
                        'Points':[0],
                        'Event':[0],
                        'Gender':[0],
                        'Year':[0]})

# this for loop looks through the pages of the different years
for i in range(5,30):

    
    link  = browser.find_element_by_name('year')
    link.click()
    j = 2020 - i
    year = str(j)
    link.send_keys(year)
    link.send_keys(Keys.RETURN)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    soup_link = soup.find(attrs = {'name':'ranking'})

#  stuff added by ethan
    if i > 22:
        k = 2
    else:
        k = 5
    rows = soup.select('a[name="ranking"] table:nth-child(2) tr:nth-child(n+2)')
    rows = rows[0].find_all('a',recursive = True)[:k]
   
    
    for row in rows:
        soup_link = row
        gender,event = soup_link.text.split()
        ids = []
        firstnames = []
        lastnames = []
        ranks = []
        countries = []
        points = []
        link = browser.find_element_by_xpath(xpath_soup(soup_link))
        #move tothe link page
        link.click()
        #actual data collection here
        time.sleep(1)
        data_soup = BeautifulSoup(browser.page_source,'html.parser')
        tags = data_soup.find_all('tr')
        tags = tags[1:]



        #getting the data from each row and appending them to a list
        for rank in range(len(tags)):
            tag = tags[rank]
            ids.append(tag.attrs['id'])
            ranks.append(rank+1)
            lastnames.append(tag.td.next_sibling.a.text)
            firstnames.append(tag.td.next_sibling.next_sibling.a.text)
            points.append(float(tag.td.next_sibling.next_sibling.next_sibling.next_sibling.text))

    #going back to Year page
        browser.back()
        #move to next link

        soup_link = soup_link.next_sibling.next_sibling

        #forming the dataframe
        df = pd.DataFrame(list(zip(ids,ranks,lastnames,firstnames,points)),columns = ['ID','Rank','Last Name','First Name', 'Points' ])
        df['Event'] = event
        df['Gender'] = gender
        df['Year'] = j
        big_df = pd.concat([big_df,df])
        
    

big_df.to_csv('Rankings.csv')
browser.close()

KeyboardInterrupt: 

In [29]:
https://www.digitalrock.de/egroupware/ranking/sitemgr/digitalrock/eliste.html#!cat=ICC_F&cup=08_WC

SyntaxError: invalid syntax (<ipython-input-29-22b4049453b4>, line 1)

In [15]:
# scraping for the world cup events
browser = webdriver.Chrome()
browser.get("https://www.digitalrock.de/icc_calendar.php")
time.sleep(2)
soup = BeautifulSoup(browser.page_source, "html.parser")
link  = browser.find_element_by_name('year')
big_df = pd.DataFrame()
for i in range(1,30):
    df = pd.DataFrame()
    link  = browser.find_element_by_name('year')
    link.click()
    j = 2020 - i
    year = str(j)
    link.send_keys(year)
    link.send_keys(Keys.RETURN)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    soup_link = soup.find(attrs = {'name':'ranking'})
    if i > 22:
        k = 2
    else:
        k = 6
    rows = soup.select('a[name="ranking"] table:nth-child(2) tr:nth-child(n+2)')
    rows = rows[0].find_all('a',recursive = True)[:k]
    print(year)
    year_df = pd.DataFrame()
    for page in rows:
        soup_link = page
        gender,event = soup_link.text.split()
        link = browser.find_element_by_xpath(xpath_soup(soup_link))
        #move tothe link page
        link.click()
        #actual data collection here
        time.sleep(3)
#         link = browser.find_element_by_class_name('seeAlso')
        soup = BeautifulSoup(browser.page_source,"html.parser")
        time.sleep(3)
        page_link = soup.find(attrs = {'class':'seeAlso'}).li.a
        link = browser.find_element_by_xpath(xpath_soup(page_link))
        link.click()
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source,"html.parser")
        #navigate to table
        Location = soup.find(attrs = {'id':'table'})
        element = browser.find_element_by_xpath(xpath_soup(Location))
#         element = browser.find_element_by_class_name("DrWidget Resultlist")
        df_list = pd.read_html(browser.page_source)
        df_test = pd.DataFrame(df_list[0])
        df_test[year] = int(year)
        df_test['event'] = event
        df_test['gender'] = gender
        time.sleep(1)
        browser.back()
        time.sleep(3)
        browser.back()
        df_test.to_csv(f'Events_data\ {year}_{gender}_{event}.csv')
        
        
        

#         Location = soup.find(attrs = {'id':'table'}).table.thead.tr.th
#         events = []
#         Location = Location.next_sibling.next_sibling.next_sibling
#         cont = True
#         while cont:
#             try: 
#                 Location = Location.next_sibling
#                 #getting the event names
#                 events.append(Location.attrs['class'][0])
#             except AttributeError:
#                 cont = False
#         event_names = events.copy()
#         Location = soup.find(attrs = {'id':'table'}).table.tbody.tr
#         cont = True
#         while cont:
#             try:
#                 subloc = Location.td
#             except:
#                 break
#             ranks.append(subloc.text)
#             subloc = subloc.next_sibling
#             last_name.append(subloc.a.text)
#             subloc = subloc.next_sibling
#             first_name.append(subloc.a.text)
#             subloc = subloc.next_sibling.next_sibling
#             points.append(subloc.text)
#             #have now reached the events bit
#             my_dict = {}
#             for i in range(len(events)):
#                 my_dict.update({f'{events[i]}':[]})
#             for i in range(len(events)):
#                 try:
#                     subloc = subloc.next
#                     try:
#                         print(subloc.text)
#                         my_dict.setdefault(events[i],[]).append(subloc.text)
# #                         print(my_dict[events[i]])
#                     except AttributeError:
#                         print('yikes')
#                         my_dict.setdefault(events[i],[]).append(np.NaN)
#                 except:
#                     break
#             try:
#                 Location = Location.next_sibling
#             except:
#                 cont = False
        
            

        
        
        
        
# #      #forming the dataframe
#         df = pd.DataFrame(list(zip(ranks,first_name,last_name,points)),columns = ['Rank','Last Name','First Name', 'Points' ])
#         df['Event'] = event
#         df['Gender'] = gender
#         for i in range(len(events)):
#             print(len(my_dict[events[i]]))
#             df[events[i]] = my_dict[events[i]]
#         df['Year'] = j
#         df.to_csv(f'{year}_{event}_{gender}.csv')                
        
        
            
        
        
        
        #arrive at the scores per event page
        
        
        
big_df.to_csv('yearly_events.csv')      
browser.close()

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991


In [ ]:
big_df.Name

In [50]:
df_test

,Rank,Name,Name.1,Nation,Points,W-Bi 05.12.1991,W-Nü 31.10.1991,W-To 16.10.1991,W-In 20.09.1991,W-Cl 30.08.1991,W-Wi 26.04.1991,1991,event,gender
0,1,Patissier,Isabelle,FRA,435.00,1. 100.00,1. 100.00,7. (43.00),4. 55.00,2. 80.00,1. 100.00,1991,lead,WOMEN
1,2,Good,Susi,SUI,380.00,2. 80.00,4. 55.00,3. 65.00,2. 80.00,1. 100.00,4. (55.00),1991,lead,WOMEN
2,3,Erbesfield,Robyn,USA,376.00,3. 65.00,2. 80.00,1. 100.00,2. 80.00,5. 51.00,5. (51.00),1991,lead,WOMEN
3,4,Raybaud,Nanette,FRA,365.00,4. 55.00,3. 65.00,2. 80.00,1. 100.00,13. (26.00),3. 65.00,1991,lead,WOMEN
4,5,Iovane,Luisa,ITA,242.00,7. 43.00,6. 47.00,8. 40.00,8. (40.00),3. 65.00,6. 47.00,1991,lead,WOMEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,78,Craciun-Pantis,Mihaela,ROU,2.00,NaN,NaN,NaN,NaN,29. 2.00,NaN,1991,lead,WOMEN
79,78,Kavoussi,Karin,AUT,2.00,NaN,NaN,NaN,29. 2.00,NaN,NaN,1991,lead,WOMEN
80,78,Lukancic,Metka,SLO,2.00,NaN,NaN,NaN,29. 2.00,NaN,NaN,1991,lead,WOMEN
81,78,Saulimovskaia,Margarita,RUS,2.00,NaN,NaN,NaN,29. 2.00,NaN,NaN,1991,lead,WOMEN


In [49]:
year_df

""
